<a href="https://colab.research.google.com/github/Xnretro/Colab-books/blob/main/Caso_Pr%C3%A1ctico_Unidad_1_Enunciado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1: Movie Ratings

En este ejercicio vamos a trabajar con un conjunto de datos que contiene información
sobre películas y valoraciones de usuarios.

El fichero movies.csv contiene las siguientes

columnas:

- movieId: identificador de la película.
- title: título de la película.
- genres: géneros de la película. Están separados por el carácter "|".

El fichero ratings.csv contiene las siguientes columnas:

- userId: identificador del usuario que ha valorado la película.
- movieId: identificador de la película.
- rating: valoración de la película por el usuario, en una escala de 0.5 a 5.
- timestamp: fecha y hora en la que se realizó la valoración. Está en formato
UNIX, es decir, el número de segundos que han pasado desde el 1 de enero de 1970. Luego veremos cómo convertirlo a un formato más legible.

## Paso 1.1 | Carga de datos

Lee los ficheros de datos y almacénalos en dos DataFrames de Pandas. Llámalos
movies y ratings. ¿Cuántas películas y valoraciones hay en cada DataFrame?

In [242]:
# importamos las librerias que usaremos
import pandas as pd
import numpy as np

In [243]:
from google.colab import drive
drive.mount("/content/Drive")

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [244]:
path_movies = "/content/Drive/MyDrive/DataSet/course/movies.csv"
path_ratings = "/content/Drive/MyDrive/DataSet/course/ratings.csv"
movies = pd.read_csv(path_movies)
ratings = pd.read_csv(path_ratings)

In [245]:
print(f"Número de películas que existen : {len(movies)}")

Número de películas que existen : 45447


In [246]:
print(f"Número de películas que existen : {movies.shape[0]}")

Número de películas que existen : 45447


In [247]:
print(f"Número de valoraciones que existen : {ratings.shape[0]}")

Número de valoraciones que existen : 500000


In [248]:
print(f"Número de valoraciones que existen : {len(ratings)}")

Número de valoraciones que existen : 500000


## Paso 1.2 | Unión de DataFrames

Añade el título y los géneros de las películas a la tabla de valoraciones de tal forma
que tengamos una única tabla con toda la información. Llámala movie_ratings. Las *películas* que no tengan valoraciones o ratings de películas que no conocemos no nos
interesan.


In [249]:
# creamos la nueva tabla uniendo las tablas correspondientes
movie_ratings = ratings.merge(movies,right_on="movieId",left_on="movieId")
movie_ratings.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,110,1.0,1425941529,Braveheart (1995),Action|Drama|War
1,1,147,4.5,1425942435,"Basketball Diaries, The (1995)",Drama
2,1,858,5.0,1425941523,"Godfather, The (1972)",Crime|Drama
3,1,1221,5.0,1425941546,"Godfather: Part II, The (1974)",Crime|Drama
4,1,1246,5.0,1425941556,Dead Poets Society (1989),Drama


In [250]:
# verificamos si existen datos dentro de nuestro dataset, esto nos ayuda a indentificar que la unión
# se hizo correctamente
movie_ratings.isna().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0
title,0
genres,0


Pregunta:

- ¿Cuantas valoraciones ha recibido la película "Dark Knight, The (2008)"? ¿Y de
5 estrellas?

In [251]:
count_raiting = len((movie_ratings[movie_ratings["title"] == "Dark Knight, The (2008)"]))
print(f"La cantidad de valoraciones que ha recibido la pelicula Dark Knight, The (2008) es de {count_raiting}")

La cantidad de valoraciones que ha recibido la pelicula Dark Knight, The (2008) es de 794


In [252]:
count_raiting = len((movie_ratings[(movie_ratings["title"] == "Dark Knight, The (2008)") & (movie_ratings["rating"] >= 5)]))
print(f"La cantidad de valoraciones con 5 estrellas que ha recibido la pelicula Dark Knight, The (2008)  es de {count_raiting}")

La cantidad de valoraciones con 5 estrellas que ha recibido la pelicula Dark Knight, The (2008)  es de 240


## Paso 1.3 | Calcula la valoracion media y el número de valoraciones de cada película

Almacena el resultado en un nuevo DataFrame llamado movie_stats. La funcion
size nos da el número de valoraciones, y mean la valoración media.


In [253]:
movie_stats = movie_ratings.groupby(["movieId"])["rating"].aggregate(["mean","size"]).merge(movies,left_index=True,right_on="movieId")
movie_stats.head(10)

,mean,size,movieId,title,genres
0,3.887444,1346,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3.218690,519,2,Jumanji (1995),Adventure|Children|Fantasy
2,3.165574,305,3,Grumpier Old Men (1995),Comedy|Romance
3,2.851852,54,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,3.057190,306,5,Father of the Bride Part II (1995),Comedy
5,3.867978,534,6,Heat (1995),Action|Crime|Thriller
6,3.331761,318,7,Sabrina (1995),Comedy|Romance
7,3.250000,34,8,Tom and Huck (1995),Adventure|Children
8,2.988506,87,9,Sudden Death (1995),Action
9,3.426984,630,10,GoldenEye (1995),Action|Adventure|Thriller


## Paso 1.4 | ¿Qué película tiene la valoración media más alta?

Para tener un resultado más fiable filtra aquellas películas que tengan al menos 100
valoraciones y extrae el top 10 de películas con la valoración media más alta.


In [254]:
df_filter = (movie_stats[movie_stats["size"] >= 100].sort_values("mean",ascending=False).head(10)).merge(movies,left_index=True,right_on="movieId").reset_index(drop=True)
df_filter

,movieId,mean,size,movieId_x,title_x,genres_x,movieId_y,title_y,genres_y
0,315,4.421611,1837,318,"Shawshank Redemption, The (1994)",Crime|Drama,315,"Specialist, The (1994)",Action|Drama|Thriller
1,1226,4.371681,113,1254,"Treasure of the Sierra Madre, The (1948)",Action|Adventure|Drama|Western,1226,"Quiet Man, The (1952)",Drama|Romance
2,3347,4.366972,109,3435,Double Indemnity (1944),Crime|Drama|Film-Noir,3347,Never Cry Wolf (1983),Adventure|Drama
3,843,4.361326,1161,858,"Godfather, The (1972)",Crime|Drama,843,Lotto Land (1995),Drama
4,49,4.309783,1196,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,49,When Night Is Falling (1995),Drama|Romance
5,1936,4.283784,259,2019,Seven Samurai (Shichinin no samurai) (1954),Action|Adventure|Drama,1936,Mrs. Miniver (1942),Drama|War
6,887,4.279012,405,904,Rear Window (1954),Mystery|Thriller,887,Talk of Angels (1998),Drama
7,1169,4.274905,793,1193,One Flew Over the Cuckoo's Nest (1975),Drama,1169,American Dream (1990),Documentary
8,1186,4.270992,131,1212,"Third Man, The (1949)",Film-Noir|Mystery|Thriller,1186,"Sex, Lies, and Videotape (1989)",Drama
9,1256,4.268182,110,1284,"Big Sleep, The (1946)",Crime|Film-Noir|Mystery,1256,Duck Soup (1933),Comedy|Musical|War


## Paso 1.5 | Extracción del año de la película

El año de la película se encuentra entre paréntesis en el título, añade una columna al
DataFrame movies con el año de la película asumiendo que siempre se encuentra en
la misma posición. Utiliza una función lambda para extraer el año del título. Una vez
que lo hayas extraído, elimina el año del título.

- El enunciado nos da la opción de usar una función lambda. Sin embargo, he considerado usar una expresión regular porque creo que sería una opción más óptima.

In [255]:
regex = r"\((\d{4})\)$"
movies["year"] = movies["title"].str.extract(regex)
movies["title"] = movies["title"].str.replace(regex, "", regex=True)
movies.head(5)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Si analizamos la columna 'year' que hemos creado, podemos ver que tiene valores nulos, lo que podría afectar nuestro análisis más adelante.

In [256]:
movies["year"].isna().sum()

np.int64(2)

Revisando aquellos valores nulos que no se han podido convertir a un año, vemos que no tienen la estructura adecuada para su conversión.

In [257]:
movies[movies["year"].isna()]

,movieId,title,genres,year
38957,159257,6A,(no genres listed),NaN
43633,171749,Death Note: Desu nôto (2006–2007),(no genres listed),NaN


En un análisis, tratar los valores faltantes es muy común, pero para nuestro ejemplo, una solución rápida es eliminarlos de nuestro conjunto de datos

In [258]:
movies = movies.dropna(subset=["year"])
movies.shape

(45445, 4)

Verificamos nuevamente

In [259]:
movies["year"].isna().sum()

np.int64(0)

Ahora convertimos nuestro campo 'year' a tipo entero, para un análisis correcto más adelante

In [260]:
movies["year"] = movies["year"].astype(int)

In [261]:
movies["year"].dtype

dtype('int64')

## Paso 1.6 | Filtro por género

¿Cómo podríamos filtrar las películas de un género concreto? Por ejemplo, ¿cómo
podríamos obtener las películas de acción de 1993? Piensa que la columna genres
es una cadena de texto y por tanto podemos utilizar el método str.contains para
filtrar por género.

In [262]:
movies.head(5)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [263]:
df_filter = movies[(movies["genres"].str.upper().str.contains("ACTION")) & (movies["year"] == 1995)]
df_filter.head(5)

,movieId,title,genres,year
5,6,Heat,Action|Crime|Thriller,1995
8,9,Sudden Death,Action,1995
9,10,GoldenEye,Action|Adventure|Thriller,1995
14,15,Cutthroat Island,Action|Adventure|Romance,1995
19,20,Money Train,Action|Comedy|Crime|Drama|Thriller,1995


## Paso 1.7 | Tratamiento columna de fecha UNIX

El campo timestamp de la tabla `ratings` está en formato UNIX, esto quiere decir que
se trata de un número entero que representa los segundos que han pasado desde el 1
de enero de 1970. Para convertirlo a un formato más legible podemos utilizar la
función to_datetime de Pandas. Crea una nueva columna en la tabla ratings llamada
date que contenga la fecha en un formato legible. ¿Cuando fue la primera y la última
valoración?

In [264]:
ratings["date"] = pd.to_datetime(ratings["timestamp"],unit="s")
ratings.head(10)

,userId,movieId,rating,timestamp,date
0,1,110,1.0,1425941529,2015-03-09 22:52:09
1,1,147,4.5,1425942435,2015-03-09 23:07:15
2,1,858,5.0,1425941523,2015-03-09 22:52:03
3,1,1221,5.0,1425941546,2015-03-09 22:52:26
4,1,1246,5.0,1425941556,2015-03-09 22:52:36
5,1,1968,4.0,1425942148,2015-03-09 23:02:28
6,1,2762,4.5,1425941300,2015-03-09 22:48:20
7,1,2918,5.0,1425941593,2015-03-09 22:53:13
8,1,2959,4.0,1425941601,2015-03-09 22:53:21
9,1,4226,4.0,1425942228,2015-03-09 23:03:48


La primera y la última valoraciones fueron

In [265]:
(first,last) =  ratings["date"].min(),ratings["date"].max()

In [266]:
print(f"primera valoración {first}")
print(f"última valoración {last}")

primera valoración 1996-03-27 17:20:36
última valoración 2017-08-04 04:52:46


# Ejercicio 2: Estudiantes

En este ejercicio vamos a trabajar con un conjunto de datos que contiene información
sobre estudiantes y sus notas. El fichero students-mat.csv ha sido extraído del
repositorio de Machine Learning de la UCI. Para consultar la descripción completa
del conjunto de datos, puedes visitar el siguiente enlace:
https://archive.ics.uci.edu/ml/datasets/Student+Performance)

## Paso 2.1 | Carga de Datos

In [267]:
path_student = "/content/Drive/MyDrive/DataSet/course/student-mat.csv"
students = pd.read_csv(path_student,sep=";")
students.head(6)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


## Paso 2.1 | Filtro estudiantes mayores de edad

En nuestro estudio vamos a considerar solo a los estudiantes menores de 18 años,
sobrescribe el DataFrame students para que contendrá solo a los estudiantes
menores de edad.


In [268]:
students = students[students["age"] < 18]
students.head(7)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15
6,GP,M,16,U,LE3,T,2,2,other,other,...,4,4,4,1,1,3,0,12,12,11
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,4,1,4,1,1,1,6,6,5,6


## Paso 2.2 | Renombrar columnas

Las columnas G1, G2 y G3 contienen las notas de los estudiantes en tres cursos.
Renombra estas columnas a grade1, grade2 y grade3 respectivamente. Para ello,
puedes utilizar el método rename de Pandas


In [269]:
students.rename(columns={"G1":"grade1","G2":"grade2","G3":"grade3"},inplace=True)

In [270]:
students.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,grade1,grade2,grade3
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
5,GP,M,16,U,LE3,T,4,3,services,other,...,5,4,2,1,2,5,10,15,15,15


## Paso 2.3 | Filtrado de columnas

Vamos a reducir el tamaño del dataframe a solo las columnas que nos interesan.
Mantén las siguientes columnas: school, sex, age, activities y las renombradas
grade1, grade2, grade3. Sobrescribe el DataFrame students con el resultado.

In [271]:
students = students[["school","sex","age","activities","grade1","grade2","grade3"]]

In [272]:
students.head(5)

,school,sex,age,activities,grade1,grade2,grade3
1,GP,F,17,no,5,5,6
2,GP,F,15,no,7,8,10
3,GP,F,15,yes,15,14,15
4,GP,F,16,no,6,10,10
5,GP,M,16,yes,15,15,15


## Paso 2.4 | Cálculo de aprobados por curso

Primero crea tres columnas de tipo booleano (grade1_passed, grade2_passed,
grade3_passed) que indiquen si el estudiante ha aprobado cada uno de los cursos.
Un estudiante aprueba si su nota es mayor o igual a 10.
Ahora puedes contar el número de estudiantes que han aprobado cada curso.
¿Cuántos estudiantes han aprobado los tres cursos? ¿Y cuántos no han aprobado
ninguno?

In [273]:
students["grade1_passed"] = (students["grade1"] >= 10)
students["grade2_passed"] = (students["grade2"] >= 10)
students["grade3_passed"] = (students["grade3"] >= 10)
students.head(5)

,school,sex,age,activities,grade1,grade2,grade3,grade1_passed,grade2_passed,grade3_passed
1,GP,F,17,no,5,5,6,False,False,False
2,GP,F,15,no,7,8,10,False,False,True
3,GP,F,15,yes,15,14,15,True,True,True
4,GP,F,16,no,6,10,10,False,True,True
5,GP,M,16,yes,15,15,15,True,True,True


In [274]:
students[(students["grade1_passed"]) & (students["grade2_passed"]) & (students["grade3_passed"])].head(5)

,school,sex,age,activities,grade1,grade2,grade3,grade1_passed,grade2_passed,grade3_passed
3,GP,F,15,yes,15,14,15,True,True,True
5,GP,M,16,yes,15,15,15,True,True,True
6,GP,M,16,no,12,12,11,True,True,True
8,GP,M,15,no,16,18,19,True,True,True
9,GP,M,15,yes,14,15,15,True,True,True


In [275]:
count_students = len(students[(students["grade1_passed"]) & (students["grade2_passed"]) & (students["grade3_passed"])])
print(f"estudiantes que aprobaron los tres cursos {count_students}")

estudiantes que aprobaron los tres cursos 167


In [276]:
students[(~students["grade1_passed"]) & (~students["grade2_passed"]) & (~students["grade3_passed"])].head(5)

,school,sex,age,activities,grade1,grade2,grade3,grade1_passed,grade2_passed,grade3_passed
1,GP,F,17,no,5,5,6,False,False,False
7,GP,F,17,no,6,5,6,False,False,False
18,GP,M,17,yes,6,5,5,False,False,False
25,GP,F,16,no,6,9,8,False,False,False
35,GP,F,15,yes,8,7,6,False,False,False


In [277]:
count_students = len(students[(~students["grade1_passed"]) & (~students["grade2_passed"]) & (~students["grade3_passed"])])
print(f"estudiantes que no aprobaron ningun curso {count_students}")

estudiantes que no aprobaron ningun curso 61


## Step 2.5 | Nota media por estudiante
Calcula la nota media de cada estudiante y añade una nueva columna al DataFrame
llamada final_grade

In [278]:
students["final_grade"] = students.apply(lambda x:x[["grade1","grade2","grade3"]].mean(),axis=1)
students.head(6)

,school,sex,age,activities,grade1,grade2,grade3,grade1_passed,grade2_passed,grade3_passed,final_grade
1,GP,F,17,no,5,5,6,False,False,False,5.333333
2,GP,F,15,no,7,8,10,False,False,True,8.333333
3,GP,F,15,yes,15,14,15,True,True,True,14.666667
4,GP,F,16,no,6,10,10,False,True,True,8.666667
5,GP,M,16,yes,15,15,15,True,True,True,15.000000
6,GP,M,16,no,12,12,11,True,True,True,11.666667


## Paso 2.6 | Nota media final por edad y sexo
Calcula la nota media final de los estudiantes por edad y sexo, redondea el resultado
a dos decimales. Almacena el resultado en un DataFrame llamado
final_grade_stats y ordena el resultado por nota media de mayor a menor.

In [279]:
final_grade_stats = students.groupby(["age","sex"])["final_grade"].mean()
final_grade_stats.head(6)

age  sex
15   F       9.798246
     M      12.568182
16   F      10.487654
     M      11.660000
17   F      10.821839
     M      10.158333
Name: final_grade, dtype: float64

In [280]:
final_grade_stats = final_grade_stats.sort_values(ascending=False)
final_grade_stats

,,final_grade
age,sex,
15,M,12.568182
16,M,11.660000
17,F,10.821839
16,F,10.487654
17,M,10.158333
15,F,9.798246


## Paso 2.7 | Valores duplicados
¿Hay alguna fila duplicada en el DataFrame students? ¿Crees que tiene sentido que
haya filas duplicadas en este DataFrame? ¿Las eliminarías?
El método drop_duplicates de Pandas nos permite eliminar filas duplicadas pero
¿cómo sabemos si hay filas duplicadas? Puedes utilizar el método duplicated para
comprobar si hay filas duplicadas en un DataFrame.

Análisis de filas duplicadas en el DataFrame students

¿Hay filas duplicadas?
- Sí, se han identificado 15 filas duplicadas en el DataFrame.

¿Tiene sentido que existan filas duplicadas?

- Sí, tiene mucho sentido que haya filas duplicadas en el DataFrame. Cada fila representa a un estudiante, y es común que distintos estudiantes compartan características idénticas, como el sexo, la edad o las notas. Por lo tanto, la existencia de duplicados no es inusual. No obstante, si el DataFrame incluyera una columna de identificación única (ID) para cada estudiante, el caso sería diferente, ya que esos registros duplicados sí indicarían un error.

¿Las eliminarías?

- La decisión de eliminar o no los duplicados siempre dependerá del objetivo del análisis. En este caso particular, y dado que cada fila corresponde a un estudiante único, no las eliminaría. Conservar estos registros es crucial para no perder información relevante sobre cada uno de los estudiantes que forman parte del conjunto de datos.

In [281]:
len(students[students.duplicated(keep="first")])

15

In [282]:
students[students.duplicated(keep="first")]

,school,sex,age,activities,grade1,grade2,grade3,grade1_passed,grade2_passed,grade3_passed,final_grade
57,GP,M,15,yes,14,15,15,True,True,True,14.666667
103,GP,F,15,no,7,6,6,False,False,False,6.333333
115,GP,M,16,yes,15,15,16,True,True,True,15.333333
116,GP,M,15,yes,11,13,14,True,True,True,12.666667
147,GP,F,15,no,10,11,11,True,True,True,10.666667
152,GP,F,15,yes,10,10,10,True,True,True,10.000000
169,GP,F,16,no,14,14,14,True,True,True,14.000000
187,GP,M,16,yes,15,15,15,True,True,True,15.000000
201,GP,F,16,yes,8,10,10,False,True,True,9.333333
208,GP,F,16,no,9,9,10,False,False,True,9.333333


In [283]:
students.shape[0]

284